### Day 9: Disk Fragmenter

Link: https://adventofcode.com/2024/day/9#part2

Part two of the problem requires comparing each file block, starting from the end of the list, with each block that comes before it, starting from the beginning of the list, to find one free space block that fits the whole file. This results in an `O(n²)` solution, so the previous approach of creating an expanded list no longer works in reasonable time. To address this, we need to represent the blocks in a more compact way. Given the input size of 20k elements, we can achieve manageable performance with a list of blocks of similar size. To do this, each block is represented as a tuple of two values:

1. The file ID, or "." if it's a free space block.
2. The number of blocks that the value spans.

Then, we attempt to move the file blocks at the end of the list to free space blocks at the beginning, as described in the problem. Note that this process might increase the length of the list if the free space is greater than the file size being moved, requiring the insertion of another tuple to indicate the remaining free space.

Finally, calculating the checksum should be `O(n)`. We can simplify the sum of the products of consecutive indices and file IDs by using an arithmetic progression sum. For example, given file ID `3` spanning indexes `5` to `7`, instead of calculating `3 * 5 + 3 * 6 + 3 * 7`, we do `3 * (5 + 6 + 7)` or `3 * ((5 + 7) * 3 // 2)`.

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
lines: list[str] = []


with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
blocks: list[tuple[int | str, int]] = []
file_id = 0
is_free_space = False


for digit in lines[0].strip():
    block_value = "." if is_free_space else file_id
    is_free_space = not is_free_space
    digit = int(digit)

    if digit > 0:
        blocks.append((block_value, digit))

    if is_free_space:
        file_id += 1


def remove_trailing_free_space() -> None:
    while blocks and blocks[-1][0] == ".":
        blocks.pop()


remove_trailing_free_space()
idx_block = len(blocks) - 1


while idx_block > 0:
    current_block_value, current_block_size = blocks[idx_block]

    if current_block_value == ".":
        idx_block -= 1
        continue

    for idx_search in range(idx_block):
        search_block_value, search_block_size = blocks[idx_search]

        if search_block_value != ".":  # Not free space
            continue

        if search_block_size < current_block_size:  # Not enough free space to move file in
            continue

        blocks[idx_search] = (current_block_value, current_block_size)
        blocks[idx_block] = (".", current_block_size)

        if search_block_size > current_block_size:  # Recalculate free space
            blocks.insert(idx_search + 1, (".", search_block_size - current_block_size))
            idx_block += 1  # Compensate for list increasing in size

        break

    idx_block -= 1


remove_trailing_free_space()
checksum = 0
current_idx = 0


for block_value, block_size in blocks:
    next_idx = current_idx + block_size

    if block_value == ".":
        current_idx = next_idx
        continue

    sum_indexes = (next_idx + current_idx - 1) * block_size // 2
    checksum += block_value * sum_indexes
    current_idx = next_idx


print(checksum)